In [1]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [129]:
#計測地点のosm_idを指定
start_osm = 668319081
mid_osm = [385509583]
end_osm = 385509577
start_mesh = 50325641322
end_mesh = 50325633332

In [130]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [131]:
getdbname = "kddi_12_new_param"
dbname = "kddi_trkn00_monthly_anbun_3a_26"
trknName = "trkn_12_grp"
sttime = "2022-04-01 12:00:00"

In [132]:
datedir = "230225_1"
f = open(datedir+'/'+ dbname + '.csv', 'w')
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        #cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        #cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        #cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            #cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            #cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        #cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        #print(sql)
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')
        trknStParam

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        #cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                #print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #cur.execute(sql)
                #connection.commit()

        #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
        #cur.execute(sql)
        #connection.commit()
        #cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)
f.close()

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値55.766666666666666距離0.0 km
存在メッシュ：50325641144値56.41595520104869距離0.11554019701202542 km
存在メッシュ：50325642232値64.00183531720279距離1.4654394124113244 km
存在メッシュ：50325642221値65.08152036127076距離1.657568205847179 km
存在メッシュ：50325642233値63.11429248378404距離1.3075021188975637 km
存在メッシュ：50325642322値62.27229991501563距離1.1576704460331162 km
存在メッシュ：50325642223値64.92169283980601距離1.6291270701422988 km
存在メッシュ：50325642411値63.08550406985393距離1.3023792515727701 km
存在メッシュ：50325642214値64.12888400524714距離1.488047589930948 km
存在メッシュ：50325642234値63.924636772811795距離1.4517020143892478 km
存在メッシュ：50325632443値65.28305474536582距離1.693431032941764 km
存在メッシュ：50325642231値63.19990145761898距離1.3227361438019158 km
存在メッシュ：50325642321値61.45909575978846距離1.0129616404242632 km
存在メッシュ：50325633331値67.03363995427735距離2.004945789807886 km
存在メッシュ：50325633332値67.8距離2.141318736465901 km
存在メッシュ：50325632442値66.27432868431882距離1.8698271652229634 km
存在メッシュ：5032564

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値60.06666666666667距離0.0 km
存在メッシュ：50325641144値60.882324146839096距離0.11554019701202542 km
存在メッシュ：50325642232値70.41195470826814距離1.4654394124113244 km
存在メッシュ：50325642221値71.76829035227044距離1.657568205847179 km
存在メッシュ：50325642233値69.29699438983212距離1.3075021188975637 km
存在メッシュ：50325642322値68.23925580275048距離1.1576704460331162 km
存在メッシュ：50325642223値71.56750979552731距離1.6291270701422988 km
存在メッシュ：50325642411値69.26082944324679距離1.3023792515727701 km
存在メッシュ：50325642214値70.57155742302747距離1.488047589930948 km
存在メッシュ：50325642234値70.31497537341686距離1.4517020143892478 km
存在メッシュ：50325632443値72.02146443312114距離1.693431032941764 km
存在メッシュ：50325642231値69.40453918106245距離1.3227361438019158 km
存在メッシュ：50325642321値67.21768216176565距離1.0129616404242632 km
存在メッシュ：50325633331値74.22060679390059距離2.004945789807886 km
存在メッシュ：50325633332値75.18333333333334距離2.141318736465901 km
存在メッシュ：50325632442値73.26673515698593距離1.8698271652229634 km
存在

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値57.61666666666667距離0.0 km
存在メッシュ：50325641144値58.33609994576032距離0.11554019701202542 km
存在メッシュ：50325642232値66.74150727667899距離1.4654394124113244 km
存在メッシュ：50325642221値67.93783419531387距離1.657568205847179 km
存在メッシュ：50325642233値65.75808031444215距離1.3075021188975637 km
存在メッシュ：50325642322値64.8251245595741距離1.1576704460331162 km
存在メッシュ：50325642223値67.76073998870473距離1.6291270701422988 km
存在メッシュ：50325642411値65.72618179485201距離1.3023792515727701 km
存在メッシュ：50325642214値66.88228144625722距離1.488047589930948 km
存在メッシュ：50325642234値66.65596872333717距離1.4517020143892478 km
存在メッシュ：50325632443値68.16114099209508距離1.693431032941764 km
存在メッシュ：50325642231値65.85293790317893距離1.3227361438019158 km
存在メッシュ：50325642321値63.92406732386534距離1.0129616404242632 km
存在メッシュ：50325633331値70.10084759476715距離2.004945789807886 km
存在メッシュ：50325633332値70.95距離2.141318736465901 km
存在メッシュ：50325632442値69.25950546738927距離1.8698271652229634 km
存在メッシュ：503256422

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値65.26666666666667距離0.0 km
存在メッシュ：50325641144値66.16415968233599距離0.11554019701202542 km
存在メッシュ：50325642232値76.64990532765704距離1.4654394124113244 km
存在メッシュ：50325642221値78.14232315865405距離1.657568205847179 km
存在メッシュ：50325642233値75.42308019226658距離1.3075021188975637 km
存在メッシュ：50325642322値74.2592178880687距離1.1576704460331162 km
存在メッシュ：50325642223値77.92139813590914距離1.6291270701422988 km
存在メッシュ：50325642411値75.38328678907787距離1.3023792515727701 km
存在メッシュ：50325642214値76.82552110420589距離1.488047589930948 km
存在メッシュ：50325642234値76.54319598236312距離1.4517020143892478 km
存在メッシュ：50325632443値78.42089838763862距離1.693431032941764 km
存在メッシュ：50325642231値75.5414150342157距離1.3227361438019158 km
存在メッシュ：50325642321値73.135148986522距離1.0129616404242632 km
存在メッシュ：50325633331値80.84068237447201距離2.004945789807886 km
存在メッシュ：50325633332値81.9距離2.141318736465901 km
存在メッシュ：50325632442値79.79110807056813距離1.8698271652229634 km
存在メッシュ：50325642221値7

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値53.833333333333336距離0.0 km
存在メッシュ：50325641144値54.5824432351896距離0.11554019701202542 km
存在メッシュ：50325642232値63.334573618508664距離1.4654394124113244 km
存在メッシュ：50325642221値64.58024902253723距離1.657568205847179 km
存在メッシュ：50325642233値62.31058029407956距離1.3075021188975637 km
存在メッシュ：50325642322値61.33914011432321距離1.1576704460331162 km
存在メッシュ：50325642223値64.39584967990547距離1.6291270701422988 km
存在メッシュ：50325642411値62.27736596055632距離1.3023792515727701 km
存在メッシュ：50325642214値63.481154722582005距離1.488047589930948 km
存在メッシュ：50325642234値63.2455065998415距離1.4517020143892478 km
存在メッシュ：50325632443値64.81276722468567距離1.693431032941764 km
存在メッシュ：50325642231値62.40935075835173距離1.3227361438019158 km
存在メッシュ：50325642321値60.40091426764145距離1.0129616404242632 km
存在メッシュ：50325633331値66.83248672471795距離2.004945789807886 km
存在メッシュ：50325633332値67.71666666666667距離2.141318736465901 km
存在メッシュ：50325632442値65.95643923458576距離1.8698271652229634 km
存在

...

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値57.61666666666667距離0.0 km
存在メッシュ：50325641144値58.33609994576032距離0.11554019701202542 km
存在メッシュ：50325642232値66.74150727667899距離1.4654394124113244 km
存在メッシュ：50325642221値67.93783419531387距離1.657568205847179 km
存在メッシュ：50325642233値65.75808031444215距離1.3075021188975637 km
存在メッシュ：50325642322値64.8251245595741距離1.1576704460331162 km
存在メッシュ：50325642223値67.76073998870473距離1.6291270701422988 km
存在メッシュ：50325642411値65.72618179485201距離1.3023792515727701 km
存在メッシュ：50325642214値66.88228144625722距離1.488047589930948 km
存在メッシュ：50325642234値66.65596872333717距離1.4517020143892478 km
存在メッシュ：50325632443値68.16114099209508距離1.693431032941764 km
存在メッシュ：50325642231値65.85293790317893距離1.3227361438019158 km
存在メッシュ：50325642321値63.92406732386534距離1.0129616404242632 km
存在メッシュ：50325633331値70.10084759476715距離2.004945789807886 km
存在メッシュ：50325633332値70.95距離2.141318736465901 km
存在メッシュ：50325632442値69.25950546738927距離1.8698271652229634 km
存在メッシュ：503256422

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値65.26666666666667距離0.0 km
存在メッシュ：50325641144値66.16415968233599距離0.11554019701202542 km
存在メッシュ：50325642232値76.64990532765704距離1.4654394124113244 km
存在メッシュ：50325642221値78.14232315865405距離1.657568205847179 km
存在メッシュ：50325642233値75.42308019226658距離1.3075021188975637 km
存在メッシュ：50325642322値74.2592178880687距離1.1576704460331162 km
存在メッシュ：50325642223値77.92139813590914距離1.6291270701422988 km
存在メッシュ：50325642411値75.38328678907787距離1.3023792515727701 km
存在メッシュ：50325642214値76.82552110420589距離1.488047589930948 km
存在メッシュ：50325642234値76.54319598236312距離1.4517020143892478 km
存在メッシュ：50325632443値78.42089838763862距離1.693431032941764 km
存在メッシュ：50325642231値75.5414150342157距離1.3227361438019158 km
存在メッシュ：50325642321値73.135148986522距離1.0129616404242632 km
存在メッシュ：50325633331値80.84068237447201距離2.004945789807886 km
存在メッシュ：50325633332値81.9距離2.141318736465901 km
存在メッシュ：50325632442値79.79110807056813距離1.8698271652229634 km
存在メッシュ：50325642221値7

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値53.833333333333336距離0.0 km
存在メッシュ：50325641144値54.5824432351896距離0.11554019701202542 km
存在メッシュ：50325642232値63.334573618508664距離1.4654394124113244 km
存在メッシュ：50325642221値64.58024902253723距離1.657568205847179 km
存在メッシュ：50325642233値62.31058029407956距離1.3075021188975637 km
存在メッシュ：50325642322値61.33914011432321距離1.1576704460331162 km
存在メッシュ：50325642223値64.39584967990547距離1.6291270701422988 km
存在メッシュ：50325642411値62.27736596055632距離1.3023792515727701 km
存在メッシュ：50325642214値63.481154722582005距離1.488047589930948 km
存在メッシュ：50325642234値63.2455065998415距離1.4517020143892478 km
存在メッシュ：50325632443値64.81276722468567距離1.693431032941764 km
存在メッシュ：50325642231値62.40935075835173距離1.3227361438019158 km
存在メッシュ：50325642321値60.40091426764145距離1.0129616404242632 km
存在メッシュ：50325633331値66.83248672471795距離2.004945789807886 km
存在メッシュ：50325633332値67.71666666666667距離2.141318736465901 km
存在メッシュ：50325632442値65.95643923458576距離1.8698271652229634 km
存在

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値59.83050847457627距離0.0 km
存在メッシュ：50325641144値60.70176961157356距離0.11554019701202542 km
存在メッシュ：50325642232値70.8810384345109距離1.4654394124113244 km
存在メッシュ：50325642221値72.32983595561447距離1.657568205847179 km
存在メッシュ：50325642233値69.6900708796715距離1.3075021188975637 km
存在メッシュ：50325642322値68.56022588170514距離1.1576704460331162 km
存在メッシュ：50325642223値72.11536811781816距離1.6291270701422988 km
存在メッシュ：50325642411値69.65144055597885距離1.3023792515727701 km
存在メッシュ：50325642214値71.05152132237662距離1.488047589930948 km
存在メッシュ：50325642234値70.77744798435049距離1.4517020143892478 km
存在メッシュ：50325632443値72.60026900245369距離1.693431032941764 km
存在メッシュ：50325642231値69.80494703708214距離1.3227361438019158 km
存在メッシュ：50325642321値67.4690112069096距離1.0129616404242632 km
存在メッシュ：50325633331値74.9493276701515距離2.004945789807886 km
存在メッシュ：50325633332値75.97768361581922距離2.141318736465901 km
存在メッシュ：50325632442値73.93043026881578距離1.8698271652229634 km
存在メッシュ：

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値60.86440677966102距離0.0 km
存在メッシュ：50325641144値61.818433228215056距離0.11554019701202542 km
存在メッシュ：50325642232値72.96468005553562距離1.4654394124113244 km
存在メッシュ：50325642221値74.55110586410079距離1.657568205847179 km
存在メッシュ：50325642233値71.660576729405距離1.3075021188975637 km
存在メッシュ：50325642322値70.42340228760528距離1.1576704460331162 km
存在メッシュ：50325642223値74.31626468855262距離1.6291270701422988 km
存在メッシュ：50325642411値71.6182767243273距離1.3023792515727701 km
存在メッシュ：50325642214値73.15135793791002距離1.488047589930948 km
存在メッシュ：50325642234値72.85124904722592距離1.4517020143892478 km
存在メッシュ：50325632443値74.84722865472226距離1.693431032941764 km
存在メッシュ：50325642231値71.78636552890973距離1.3227361438019158 km
存在メッシュ：50325642321値69.2285278503119距離1.0129616404242632 km
存在メッシュ：50325633331値77.4194357726851距離2.004945789807886 km
存在メッシュ：50325633332値78.54548022598871距離2.141318736465901 km
存在メッシュ：50325632442値76.30374837661078距離1.8698271652229634 km
存在メッシュ：

In [133]:
connection.close()

In [88]:
#CreateGeojsonFile